In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# Path to audio dataset (update this path)
DATASET_PATH = "path/to/audio_dataset"

In [ ]:
# Function to convert audio to spectrogram
def audio_to_spectrogram(audio_path, img_size=(128, 128)):
    y, sr = librosa.load(audio_path, sr=None)
    S = librosa.feature.melspectrogram(y=y, sr=sr)
    S_dB = librosa.power_to_db(S, ref=np.max)

    fig, ax = plt.subplots()
    librosa.display.specshow(S_dB, sr=sr, ax=ax)
    ax.axis('off')
    fig.canvas.draw()

    img = np.array(fig.canvas.renderer.buffer_rgba())
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2GRAY)
    img = cv2.resize(img, img_size)
    plt.close(fig)
    return img

In [ ]:
# Function to load audio files and extract HOG features
def extract_features_and_labels(dataset_path, img_size=(128, 128)):
    features = []
    labels = []
    label_encoder = LabelEncoder()

    for category in os.listdir(dataset_path):
        category_path = os.path.join(dataset_path, category)
        if not os.path.isdir(category_path):
            continue

        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            if not file_path.endswith(".wav"):
                continue

            img = audio_to_spectrogram(file_path, img_size)
            hog_features = hog(img, pixels_per_cell=(16, 16), cells_per_block=(2, 2),
                               orientations=9, block_norm='L2-Hys', visualize=False)
            features.append(hog_features)
            labels.append(category)

    labels = label_encoder.fit_transform(labels)
    return np.array(features), np.array(labels), label_encoder

In [ ]:
# Load data and extract features
X, y, label_encoder = extract_features_and_labels(DATASET_PATH)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Save the trained model
import joblib
joblib.dump(svm_classifier, "svm_audio_classifier.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
